# Лабораторна робота №2

**Тема:** Математична реалізація та візуалізація лінійної регресії з градієнтним спуском

## Мета роботи
Зрозуміти та математично реалізувати три складові задачі машинного навчання: модель (лінійна регресія), функціонал якості (MSE, середньоквадратична помилка) та алгоритм оптимізації (градієнтний спуск).

Наочно продемонструвати процес зведення задачі навчання до задачі оптимізації.

Візуалізувати вплив кроку навчання (learning rate) на збіжність алгоритму.

Проаналізувати проблему перенавчання (на якісному рівні).



**Завдання 1: Формалізація та функціонал якості (Емпіричний ризик)**

Визначення моделі:Вам необхідно побудувати модель лінійної регресії
h(x) = w_1x + w_0

x - вхідна ознака

h(x) — передбачуваний результат

w_1 і w_0 — параметри моделі (ваги), які потрібно навчити.

**Крок 1.** Імпортуйте необхідні бібліотеки та згенеруйте дані

In [ ]:
# Імпорт необхідних бібліотек
import numpy as np
import matplotlib.pyplot as plt

# 1. Генерація синтетичних даних (N=100)
np.random.seed(42) # Для відтворюваності результату
N = 100
# Справжні параметри (True weights)
W_true = 2.5
B_true = 1.0

# Генеруємо вхідні дані X (від 0 до 10)
X = np.linspace(0, 10, N)
# Генеруємо істинні відповіді Y з додаванням шуму (квадратична помилка)
Y = W_true * X + B_true + np.random.normal(0, 3, N)

print(f"Кількість об'єктів (N): {N}")
print("Дані згенеровано.")

# 2. Візуалізація згенерованих даних
plt.figure(figsize=(8, 5))
plt.scatter(X, Y, label='Згенеровані дані (X, Y)', color='skyblue')
plt.title('Вибірка для лінійної регресії')
plt.xlabel('Ознака X')
plt.ylabel('Цільова змінна Y')
plt.legend()
plt.grid(True, alpha=0.5)
plt.show()

**Крок 2**. Реалізація моделі (Гіпотези)

1. Назва функції: hypothesis(X, W1, W0)

Ця функція — це наша лінійна модель або, як ми її ще називаємо, гіпотеза.

 Її завдання — передбачити значення Y (цільової змінної) для будь-якого вхідного значення X (ознаки).

 Уявіть, що модель — це пряма лінія.

 W1 (вага) визначає нахил цієї лінії.

 W0 (зсув) визначає, де ця лінія перетинає вісь Y.

 Функція просто бере ваш вхідний X, множить його на W_1 і додає W_0.

  Результатом є передбачувана координата Y, яка лежить на нашій лінії.
  
2. Реалізація функціоналу якості (Емпіричний ризик)

Назва функції: compute_cost(X, Y, W1, W0)

Ця функція є нашим вимірювачем якості або Функціоналом втрат (Cost Function).

Вона показує, наскільки "погана" або "хороша" наша поточна модель, вимірюючи загальну помилку на всіх точках даних.

Прогноз: Спочатку вона використовує нашу модель (hypothesis) для передбачення Y для кожного X.

Помилка: Далі вона обчислює різницю між тим, що спрогнозувала модель, і справжнім значенням Y у наших даних.

Це відстань від кожної точки до нашої прямої лінії.

Квадратування: Щоб уникнути взаємного скорочення позитивних і негативних помилок, ми підносимо кожну помилку до квадрату.

Це також має ефект сильнішого "покарання" моделі за великі помилки.

Середнє: Нарешті, ми підсумовуємо всі ці квадрати помилок і ділимо на кількість точок.

Результат, який ми називаємо Емпіричний ризик (J), є середньою мірою того, наскільки далеко наша пряма лінія знаходиться від усіх точок.

Мета машинного навчання — знайти такі W_1 і W_0$, які мінімізують це значення J!

In [ ]:
## Завдання 1.1: Реалізація моделі (Гіпотези)
def hypothesis(X, W1, W0):
    """
    Реалізує лінійну модель: h(x) = W1*x + W0
    """
    # TODO: Реалізуйте функцію h(x).
    return W1 * X + W0

## Завдання 1.2: Реалізація функціоналу якості (MSE / Емпіричний ризик)
def compute_cost(X, Y, W1, W0):
    """
    Обчислює середньоквадратичну помилку (MSE) / Емпіричний ризик J.
    J(W1, W0) = (1 / 2*N) * sum((h(x) - y)^2)
    """
    N = len(Y)
    predictions = hypothesis(X, W1, W0)

    # TODO: Обчисліть помилку (різницю між прогнозом та істинним Y)
    error = predictions - Y

    # TODO: Обчисліть середньоквадратичну помилку (MSE)
    # Зверніть увагу на множник (1 / 2*N) згідно з математичною формулою
    J = (1 / (2 * N)) * np.sum(error**2)

    return J

# Перевірка (тестування з початковими параметрами)
W1_init, W0_init = 0.0, 0.0
initial_cost = compute_cost(X, Y, W1_init, W0_init)
print(f"Початкове значення функціоналу якості (J) при W1={W1_init}, W0={W0_init}: {initial_cost:.4f}")

**Крок 3.** Реалізація градієнтного спуску

Функція gradient_descent – це серце нашого навчання.

Вона відповідає за пошук найкращої прямої лінії, яка мінімізує помилку (функціонал якості J).

Уявіть, що ви стоїте в горах (ваша поточна помилка J), і вам треба спуститися в долину (мінімум помилки).

1. Цикл Навчання (Епохи)

Весь процес відбувається всередині великого циклу for i in range(n_iterations).

Кожен прохід цього циклу називається епохою. Це як один крок, який ми робимо в горах.

Ми повторюємо цей процес тисячу разів, щоб переконатися, що дісталися до самого дна.

2. Визначення Напрямку (Градієнти)

На кожному кроці нам потрібно знати, куди саме йти, щоб зменшити помилку J.

Градієнт — це компас, який вказує напрямок найшвидшого зростання помилки. Ми йдемо проти градієнта! Щоб мінімізувати помилку, ми рухаємося в абсолютно протилежному напрямку.У нашому коді ми обчислюємо два «градієнти» (dW1 і dW0).

dW1 показує, як потрібно змінити нахил лінії.

dW0 показує, як потрібно змінити зсув лінії.

Ці значення розраховуються на основі поточної помилки (error) по всіх точках.

 Грубо кажучи, якщо лінія надто високо над точками, градієнт вкаже, що її треба опустити.

3. Оновлення Параметрів (Крок Спуску). Це найважливіший етап. Ми оновлюємо наші параметри (W_1 і W_0), рухаючись у напрямку, вказаному градієнтом.

 W1 = W1 - learning_rate * dW1W0 = W0 - learning_rate * dW0

Тут з'являється параметр Крок Навчання (learning_rate).

Крок Навчання (h) — це довжина кроку, який ми робимо. Якщо крок занадто великий, ми можемо перестрибнути долину або взагалі "розбігтися" і потрапити в ще більші гори (помилка зросте).Якщо крок занадто малий, ми будемо спускатися дуже довго.Ми віднімаємо градієнт, помножений на крок навчання, від поточних параметрів. Це змушує лінію поступово рухатися ближче до ідеального положення.

4. Збереження Історії

Наприкінці кожної епохи ми обчислюємо та записуємо поточне значення помилки (cost_history). Це дозволяє нам після завершення навчання побудувати графік і візуально переконатися, що наша помилка справді зменшувалася з кожним кроком і алгоритм успішно "дійшов до долини".

Після 1000 епох (ітерацій) ми отримуємо фінальні оптимальні параметри (W_1 та W_0), які є нашою найкращою можливою прямою лінією для прогнозування.

In [ ]:
def gradient_descent(X, Y, W1, W0, learning_rate, n_iterations):
    """
    Виконує повний градієнтний спуск для лінійної регресії.
    """
    N = len(Y)
    cost_history = []  # Для запису J на кожній епосі

    for i in range(n_iterations):
        # 1. Обчислення прогнозу (h(x)) та помилки
        predictions = hypothesis(X, W1, W0)
        error = predictions - Y

        # 2. Обчислення градієнтів (часткових похідних)
        # TODO: Реалізуйте обчислення градієнта для W1
        dW1 = (1 / N) * np.sum(error * X)

        # TODO: Реалізуйте обчислення градієнта для W0
        dW0 = (1 / N) * np.sum(error)

        # 3. Оновлення параметрів (крок градієнтного спуску)
        W1 = W1 - learning_rate * dW1
        W0 = W0 - learning_rate * dW0

        # 4. Запис значення функціоналу якості (Емпіричного ризику)
        cost = compute_cost(X, Y, W1, W0)
        cost_history.append(cost)

        # Друк прогресу кожні 100 епох
        if i % 100 == 0:
            print(f"Епоха {i}: J = {cost:.4f}, W1 = {W1:.4f}, W0 = {W0:.4f}")

    return W1, W0, cost_history

# Параметри навчання
LEARNING_RATE = 0.01  # Крок навчання (h)
N_ITERATIONS = 1000

# Початкові ваги
W1_start, W0_start = 0.0, 0.0

# Запуск навчання
W1_final, W0_final, cost_hist = gradient_descent(
    X, Y, W1_start, W0_start, LEARNING_RATE, N_ITERATIONS
)

print("-" * 30)
print(f"✅ Навчання завершено за {N_ITERATIONS} епох.")
print(f"Оптимальні параметри: W1 = {W1_final:.4f}, W0 = {W0_final:.4f}")
print(f"Фінальне значення J: {cost_hist[-1]:.4f}")


**Крок 4.**Аналіз та Візуалізація

Метою цього етапу є візуальна оцінка того, наскільки успішно алгоритм Градієнтного спуску мінімізував помилку та наскільки добре фінальна модель апроксимує дані.

Візуалізація збіжності (Емпіричний ризик vs. Епоха)

Завдання: Запустіть наданий код. Він візуалізує, як змінювалося значення Функціоналу якості J (Емпіричного ризику) протягом 1000 епох навчання.

Очікуваний результат: На графіку ви побачите, як крива помилки швидко падає на початкових етапах, а потім вирівнюється, наближаючись до мінімуму. Це є візуальним доказом того, що задача навчання була успішно зведена до задачі оптимізації.

Візуалізація фінальної моделі

Завдання: Запустіть наданий код. Він відобразить:

Згенеровані дані (сині точки).

Істинну модель (зелена пунктирна лінія) — та залежність, яку ми намагалися відтворити.

Фінальну модель (червона суцільна лінія), знайдену алгоритмом.

Очікуваний результат: Фінальна червона лінія повинна бути дуже близькою до істинної зеленої лінії, підтверджуючи, що параметри W_1 та W_0 були успішно навчені.

In [ ]:
## Завдання 3.1: Візуалізація збіжності (Емпіричний ризик vs. Епоха)
plt.figure(figsize=(10, 6))
# TODO: Побудуйте графік залежності cost_hist від номера епохи
plt.plot(range(N_ITERATIONS), cost_hist, color='orange', label='Емпіричний ризик Q(θ)')
plt.title('Зведення задачі навчання до оптимізації: Мінімізація J')
plt.xlabel('Епоха навчання (t)')
plt.ylabel('Функціонал якості J (MSE)')
plt.legend()
plt.grid(True, alpha=0.5)
plt.show()


## Завдання 3.2: Візуалізація фінальної моделі
plt.figure(figsize=(8, 5))
plt.scatter(X, Y, label='Згенеровані дані', color='skyblue')

# Побудова фінальної лінії регресії
Y_pred_final = hypothesis(X, W1_final, W0_final)
plt.plot(X, Y_pred_final, color='red',
         label=f'Фінальна модель: Y = {W1_final:.2f}*X + {W0_final:.2f}')

# Відображення істинної моделі для порівняння
Y_true = W_true * X + B_true
plt.plot(X, Y_true, color='green', linestyle='--', alpha=0.5,
         label=f'Істинна модель: Y = {W_true}*X + {B_true}')

plt.title('Результат лінійної регресії')
plt.xlabel('Ознака X')
plt.ylabel('Цільова змінна Y')
plt.legend()
plt.show()

## Завдання 3.3: Аналіз кроку навчання (Теоретичне питання)
print("-" * 30)
print("Аналіз гіперпараметра h (Крок навчання):")
print("1. Як виглядатиме графік J, якщо LEARNING_RATE буде занадто малим (наприклад, 0.00001)?")
print("   - Відповідь: ...")
print("2. Як виглядатиме графік J, якщо LEARNING_RATE буде занадто великим (наприклад, 1.0)?")
print("   - Відповідь: ...")

**Крок 5.** Переробіть код з кроку 4 та надайте відповіді



In [ ]:
# ВАШ КОД:

**Ваші висновки:**



**Крок 6.** Демонстрація та аналіз перенавчання (Overfitting).

Щоб продемонструвати перенавчання (Overfitting) у Colab, ми повинні перейти від простої лінійної моделі до моделі з високою складністю, наприклад, поліноміальної регресії високого ступеня.

Мета цієї частини — показати, як модель з надмірною кількістю параметрів (складністю) ідеально підганяється під навчальні дані, але стає неефективною на нових (тестових) даних.

Спершу ми імітуємо процес розбиття даних на навчальну та тестову вибірки, як це вимагає математична формалізація.


In [ ]:
# Імпорт необхідних функцій для поділу даних
from sklearn.model_selection import train_test_split

print("--- Аналіз Перенавчання ---")

# 1. Поділ даних на навчальну та тестову вибірки
# 80% для навчання (X_train, Y_train), 20% для тестування (X_test, Y_test)
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.2, random_state=42
)

# 2. Визначення двох моделей: Низька складність (Linear) та Висока складність (Polynomial)
DEGREE_LOW = 1  # Лінійна модель (Низька складність)
DEGREE_HIGH = 15 # Поліном 15-го ступеня (Висока складність - потенційне перенавчання)

# 3. Функція для обчислення фінальної помилки (Емпіричного ризику Q)
def calculate_mse(Y_true, Y_pred):
    """Обчислює Середньоквадратичну помилку (MSE)."""
    return np.mean((Y_true - Y_pred)**2)


**Крок 7.**

Використовуємо вбудовану функцію NumPy для швидкого навчання поліноміальної моделі (це спрощення, оскільки ручна реалізація ГС для полінома надто складна для цієї роботи).

In [ ]:
# --- Модель 1: Лінійна (Низька складність / Оптимум) ---
# Навчання: polyfit повертає коефіцієнти (ваги) моделі
coeffs_low = np.polyfit(X_train, Y_train, DEGREE_LOW)
# Побудова функції-гіпотези
model_low = np.poly1d(coeffs_low)

# Оцінка помилки (Емпіричний ризик Q)
Q_train_low = calculate_mse(Y_train, model_low(X_train))
Q_test_low = calculate_mse(Y_test, model_low(X_test))

print(f"✅ Модель N={DEGREE_LOW} (Оптимум):")
print(f"  Q на НАВЧАННІ (Q_train): {Q_train_low:.2f}")
print(f"  Q на КОНТРОЛІ (Q_test): {Q_test_low:.2f}")
print("-" * 30)

# --- Модель 2: Поліном 15-го ступеня (Висока складність / Перенавчання) ---
coeffs_high = np.polyfit(X_train, Y_train, DEGREE_HIGH)
model_high = np.poly1d(coeffs_high)

# Оцінка помилки (Емпіричний ризик Q)
Q_train_high = calculate_mse(Y_train, model_high(X_train))
Q_test_high = calculate_mse(Y_test, model_high(X_test))

print(f"💥 Модель N={DEGREE_HIGH} (Перенавчання):")
print(f"  Q на НАВЧАННІ (Q_train): {Q_train_high:.2f}")
print(f"  Q на КОНТРОЛІ (Q_test): {Q_test_high:.2f}")
print("-" * 30)

# --- Завдання 4.1: Аналіз Q ---
# TODO: Порівняйте Q_train_high та Q_test_high. Яка помилка значно більша?

**Крок 8:** Візуалізація перенавчання







In [ ]:
# Завдання 4.2: Візуалізація результатів
plt.figure(figsize=(10, 6))
plt.scatter(X_train, Y_train, label='Навчальна вибірка', color='skyblue', alpha=0.6)
plt.scatter(X_test, Y_test, label='Контрольна вибірка (Нові дані)', color='gold', alpha=0.8)

# Діапазон для побудови гладкої кривої
X_plot = np.linspace(0, 10, 200)

# Побудова лінійної моделі (Оптимум)
plt.plot(X_plot, model_low(X_plot), color='green', linestyle='-',
         label=f'Модель N=1 (Test Q: {Q_test_low:.2f})')

# Побудова складної моделі (Перенавчання)
plt.plot(X_plot, model_high(X_plot), color='red', linestyle='--',
         label=f'Модель N=15 (Test Q: {Q_test_high:.2f})')

plt.ylim(min(Y) - 5, max(Y) + 5)
plt.title('Демонстрація Перенавчання (Train Error vs. Test Error)')
plt.xlabel('Ознака X')
plt.ylabel('Цільова змінна Y')
plt.legend()
plt.grid(True, alpha=0.4)
plt.show()

**Ваші висновки**

Проаналізуйте числові результати та графік для Моделі N=15.

Яку умову (згідно з математичною формалізацією) виконала Модель N=15? Опишіть, як крива Моделі N=15 (червона) поводиться на графіку порівняно з Моделлю N=1 (зелена). Що робить поліном 15-го ступеня, що призводить до високої помилки на тестовій вибірці?


Тут мають бути Ваші висновки